# Python for Learning AI Week 3: Building AI Assistants with Gemini API

Welcome to Week 3 of our Python development series! This week, we're diving into the exciting world of Large Language Models (LLMs) by building our own AI assistant using Google's Gemini API. By the end of this session, you'll have created your own AI assistant and gained valuable insights into how modern AI systems work.

**Superhero Sidekick Metaphor**: Throughout this notebook, we'll compare working with LLMs to having your own superhero AI sidekick:
- The LLM is like your superhero sidekick with encyclopedic knowledge and special powers
- Prompts are like mission briefings you give to your sidekick
- API parameters are like special equipment you provide (e.g., "be more creative" or "stay focused")
- The sidekick's responses are the intelligence and assistance they provide for your mission

## What You'll Learn
1. Understanding Large Language Models and Google's Gemini
2. Setting up your API credentials securely
3. Making requests to the Gemini API
4. Customizing responses with different parameters
5. Building a simple chatbot with persistent history

## Prerequisites
- Basic Python knowledge ([covered in Week 1](../week1/week1_python_basics.ipynb))
- Understanding of APIs ([covered in Week 2](../week2/001_apis_and_networking.ipynb))
- A Google account (to access the Gemini API)

## 1. Setting Up Your Environment

Before we can start using the Gemini API, we need to set up our environment properly. This includes adding our API key to the `.env` file and installing the necessary packages.

### Installing Required Packages

To set up your environment for using the Gemini API, you'll need to install the necessary packages. Follow these steps:

1. Open a terminal window in VS Code (Terminal → New Terminal)
2. Navigate to the project root directory (where the `pyproject.toml` file is located)
3. Run the following command to install all project dependencies using UV:
   ```bash
   uv pip install -e .
   ```
   This command installs the project in "editable" mode, allowing you to modify the code without reinstalling.

4. Install the Google Generative AI library (if it's not already in the project dependencies):
   ```bash
   uv pip install google-generativeai
   ```

5. You can also check for the latest versions of packages and update them:
   ```bash
   uv pip install --upgrade google-generativeai python-dotenv
   ```

### Configuring Your Gemini API Key

To use the Gemini API, you need an API key. Follow these steps:

1. Visit [Google AI Studio](https://aistudio.google.com/) and sign in with your Google account
2. Click on "Get API key" in the side nav bar.
3. Click on "Create API key" on top right corner.
4. Provide a suitable name for the API key.
5. Create a new project in which the API key will be created.
6. Create your API key.
7. Click on the created API key you created from the table, copy the API key (Starts with `AI`).

Now, add your API key to the existing `.env` file in the project root directory by adding these lines:
```
GEMINI_API_KEY=your_api_key_here
GEMINI_MODEL=gemini-2.5-flash
```

You can change the model to any of the available models like:
- gemini-2.5-flash
- gemini-2.5-flash-lite
- gemini-2.5-pro

⚠️ **Important:** Never share your API key with others or commit it to public repositories!

Let's check if we can load the API key from the `.env` file:

In [7]:
# Load environment variables from .env file and initialize the Google Generative AI library
import os
import pathlib
from dotenv import load_dotenv
import google.generativeai as genai

# Load the environment variables from the .env file
load_dotenv()

# Get the API key
api_key = os.getenv('GEMINI_API_KEY')
# Set default model as an environment variable
# You can add this to your .env file too for persistence
GEMINI_MODEL = os.getenv('GEMINI_MODEL')

print(f"Using Gemini model: {GEMINI_MODEL}")

# Check if the API key exists and is properly formatted
if api_key and api_key.startswith('AI'):
    print("✅ API key loaded successfully!")
    # Show a masked version of the key for verification (first 4 chars and last 4 chars)
    masked_key = api_key[:4] + '*' * (len(api_key) - 8) + api_key[-4:]
    print(f"API Key: {masked_key}")
    
    # Configure the API with our key
    genai.configure(api_key=api_key)
    
    # List available models
    print("\nAvailable Gemini Models:")
    print("-" * 30)
    
    for model in genai.list_models():
        if 'generateContent' in model.supported_generation_methods:
            print(f"• {model.name}")
else:
    print("❌ API key not found or invalid.")
    print("Please add GEMINI_API_KEY=your_api_key_here to your .env file in the project root directory.")

Using Gemini model: gemini-2.5-flash
✅ API key loaded successfully!
API Key: AIza*******************************TjOc

Available Gemini Models:
------------------------------
• models/gemini-2.5-pro-preview-03-25
• models/gemini-2.5-flash-preview-05-20
• models/gemini-2.5-flash
• models/gemini-2.5-flash-lite-preview-06-17
• models/gemini-2.5-pro-preview-05-06
• models/gemini-2.5-pro-preview-06-05
• models/gemini-2.5-pro
• models/gemini-2.0-flash-exp
• models/gemini-2.0-flash
• models/gemini-2.0-flash-001
• models/gemini-2.0-flash-exp-image-generation
• models/gemini-2.0-flash-lite-001
• models/gemini-2.0-flash-lite
• models/gemini-2.0-flash-preview-image-generation
• models/gemini-2.0-flash-lite-preview-02-05
• models/gemini-2.0-flash-lite-preview
• models/gemini-2.0-pro-exp
• models/gemini-2.0-pro-exp-02-05
• models/gemini-exp-1206
• models/gemini-2.0-flash-thinking-exp-01-21
• models/gemini-2.0-flash-thinking-exp
• models/gemini-2.0-flash-thinking-exp-1219
• models/gemini-2.5-flash-pr

## 2. Making Simple Requests to Gemini

Now that we've set up our environment and connected to the API, let's start by making a simple request to the Gemini model. We'll create a function that allows us to send a prompt and receive a response.

**Superhero Sidekick Metaphor**: This is like giving your superhero sidekick a simple reconnaissance mission—perhaps asking them to scout ahead and report what they see. We're starting with something simple to test their capabilities.

In [8]:
# Create a function for basic interaction with Gemini
def ask_gemini(prompt, model_name=None):
    """
    Send a prompt to the Gemini model and get a response.
    
    Args:
        prompt (str): The prompt to send to the model
        model_name (str, optional): The name of the Gemini model to use, defaults to environment variable
        
    Returns:
        str: The model's response
    """
    # Use the model from environment variable if not specified
    if model_name is None:
        model_name = GEMINI_MODEL
        
    print(f"Using model: {model_name}")
    
    # Select the model
    model = genai.GenerativeModel(model_name)
    
    # Generate a response
    response = model.generate_content(prompt)
    
    return response.text

In [9]:
# Let's try a simple prompt
prompt = "Who are some super heroes who can code in Python?"
response = ask_gemini(prompt)

print("Prompt:", prompt)
print("\nResponse:")
print(response)

Using model: gemini-2.5-flash
Prompt: Who are some super heroes who can code in Python?

Response:
It's rare for comic books or movies to explicitly name the programming languages superheroes use! Their tech skills are usually described generically as "hacking," "advanced AI," or "proprietary algorithms."

However, if we consider their fields of expertise and the common uses of Python today, we can make some *very strong speculative cases* for which superheroes would likely be proficient in Python:

**Highly Plausible / Almost Certain:**

1.  **Tony Stark (Iron Man):**
    *   **Why Python?** He builds advanced AIs like J.A.R.V.I.S., F.R.I.D.A.Y., and U. Python is a foundational language for AI, machine learning, and data science. He'd use it for developing suit functionalities, analyzing threats, and managing his vast tech empire. Its versatility and rapid prototyping capabilities would appeal to him.

2.  **Barbara Gordon (Oracle):**
    *   **Why Python?** As one of the greatest hac

## 3. Customizing Model Behavior with Parameters

One powerful aspect of using LLMs like Gemini is that you can customize various parameters to control the behavior of the model. Let's explore the most important ones:

- **Temperature**: Controls randomness (0.0 = deterministic, 1.0 = creative)
- **Top-k**: Limits token selection to the k most likely next tokens
- **Top-p**: Limits token selection to a subset with a cumulative probability of p
- **Max output tokens**: Sets a limit on response length

**Superhero Sidekick Metaphor**: These parameters are like configuring your sidekick's approach to the mission:
- Temperature is like adjusting how creative or by-the-book your sidekick should be
- Top-k is like limiting their toolkit to only the most reliable gadgets
- Top-p is like telling them to only use strategies they're confident in
- Max output tokens is like setting a time limit for their mission report

In [13]:
# Enhanced function with customizable parameters
def ask_gemini_custom(prompt, temperature=0.7, top_k=40, top_p=0.95, model_name=None):
    """
    Send a prompt to the Gemini model with customizable parameters.
    
    Args:
        prompt (str): The prompt to send to the model
        temperature (float): Controls randomness (0.0 = deterministic, 1.0 = creative)
        top_k (int): Limits token selection to the k most likely next tokens
        top_p (float): Limits token selection to a subset with cumulative probability of p
        max_tokens (int): Maximum number of tokens to generate
        model_name (str, optional): The name of the Gemini model to use
        
    Returns:
        str: The model's response
    """
    # Use the model from environment variable if not specified
    if model_name is None:
        model_name = GEMINI_MODEL
        
    # Configure generation parameters
    generation_config = genai.types.GenerationConfig(
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
    )
    
    # Select the model with the specified configuration
    model = genai.GenerativeModel(model_name, generation_config=generation_config)
    
    # Generate a response
    response = model.generate_content(prompt)
    
    return response.text

In [ ]:
# Let's try the same prompt with different temperatures
# Rerun the below cells, to check the consistency of responses

prompt = "Write a very short poem about super heroes using respective movie references."

print("Temperature = 0.2 (More focused, less creative)")
print("-" * 50)
response_cold = ask_gemini_custom(prompt, temperature=0.2)
print(response_cold)

Temperature = 0.2 (More focused, less creative)
--------------------------------------------------
"I am Iron Man," a final snap.
Cap's "Assemble!" fills the gap.
From Gotham's night to Krypton's flight,
Heroes rise, to make things right.


In [16]:
print("Temperature = 1.0 (More creative, more diverse)")
print("-" * 50)
response_hot = ask_gemini_custom(prompt, temperature=1.0)
print(response_hot)

Temperature = 1.0 (More creative, more diverse)
--------------------------------------------------
"I am Iron Man," a final snap,
Cap's shield, no turning back.
Wakanda forever, the Dark Knight's track.


## 4. Building a Simple Chatbot with History

So far, we've been making one-off requests to the Gemini model. However, for a true chatbot experience, we need to maintain conversation history to provide context for follow-up questions. The Gemini API makes this easy with its chat functionality.

**Superhero Sidekick Metaphor**: This is like having an ongoing mission with your sidekick, where they remember all the previous intel and context, allowing for a more coordinated operation rather than treating each interaction as a brand new mission.

In [18]:
# Let's create a function that prepares a chat session for a superhero-themed conversation
def setup_hero_chat(model_name=None):
    """
    Set up a chat session for a superhero-themed conversation.
    
    Args:
        model_name (str, optional): The name of the Gemini model to use
    
    Returns:
        tuple: The chat object and model object
    """
    # Use the model from environment variable if not specified
    if model_name is None:
        model_name = GEMINI_MODEL
        
    print(f"Setting up superhero chat with model: {model_name}")
    
    # Select the model
    model = genai.GenerativeModel(model_name)
    
    # Start a chat session
    chat = model.start_chat(history=[])
    
    print("Superhero chat session initialized and ready for your questions!")
    print("This chat maintains context between questions, like a true sidekick.")
    print("-" * 50)
    
    return chat, model

# Initialize our superhero chat session
hero_chat, hero_model = setup_hero_chat()

Setting up superhero chat with model: gemini-2.5-flash
Superhero chat session initialized and ready for your questions!
This chat maintains context between questions, like a true sidekick.
--------------------------------------------------


In [19]:
# Question 1: Ask about scientist superheroes
question1 = "Who are the most popular superheroes that are also scientists?"
print(f"Question: {question1}\n")

# Send the message and get the response
try:
    response1 = hero_chat.send_message(question1)
    print(f"Gemini: {response1.text}")
except Exception as e:
    print(f"Error: {str(e)}")

Question: Who are the most popular superheroes that are also scientists?

Gemini: You're looking for characters who combine incredible intellect and scientific prowess with their superheroic identities! This is a classic trope, as science often explains (or creates) their powers.

Here are the most popular superheroes who are also prominent scientists:

1.  **Bruce Banner (The Hulk):** A brilliant nuclear physicist who was exposed to gamma radiation during an experiment, transforming him into the monstrous Hulk. His scientific background is central to his origin and his ongoing struggle to control his alter ego.

2.  **Tony Stark (Iron Man):** An unparalleled genius inventor, engineer, and futurist. His entire superhero persona is built around the advanced technology he designs and builds, from his suits to his AI systems like J.A.R.V.I.S. and F.R.I.D.A.Y.

3.  **Peter Parker (Spider-Man):** From a brilliant high school student specializing in chemistry, physics, and biology, Peter gro

In [20]:
# Question 2: Ask about Iron Man's scientific background
question2 = "Can you describe the scientific background of Tony Stark/Iron Man?"
print(f"Question: {question2}\n")

# Send the message and get the response
try:
    response2 = hero_chat.send_message(question2)
    print(f"Gemini: {response2.text}")
except Exception as e:
    print(f"Error: {str(e)}")

Question: Can you describe the scientific background of Tony Stark/Iron Man?

Gemini: Tony Stark's scientific background is incredibly deep and broad, making him one of the most brilliant minds in the Marvel Universe. He's not just an inventor; he's a true polymath with a prodigious understanding of numerous scientific and engineering disciplines.

Here's a breakdown of his scientific background:

1.  **Prodigious Intellect & Early Education:**
    *   Tony was a child prodigy, graduating from MIT with a double major in Electrical Engineering and Physics by the age of 15 or 17 (depending on the continuity, but always remarkably young).
    *   He inherited not only his father Howard Stark's industrial empire but also a significant portion of his genius, often surpassing Howard's theoretical work with practical application.

2.  **Core Scientific & Engineering Disciplines:**

    *   **Electrical Engineering:** This is fundamental to almost everything he does. He designs complex circuit

In [21]:
# Question 3: Ask about Black Panther's vibranium technology
question3 = "What scientific principles does Black Panther's vibranium technology use?"
print(f"Question: {question3}\n")

# Send the message and get the response
try:
    response3 = hero_chat.send_message(question3)
    print(f"Gemini: {response3.text}")
except Exception as e:
    print(f"Error: {str(e)}")

Question: What scientific principles does Black Panther's vibranium technology use?

Gemini: Black Panther's vibranium technology is arguably the most advanced on Earth within the Marvel Cinematic Universe, pushing the boundaries of what we understand about material science, energy, and information technology. While fictional, it operates on fascinating (and often speculative) scientific principles:

1.  **Kinetic Energy Manipulation (Absorption, Storage, and Redirection):**
    *   **Principle:** This is vibranium's most famous and fundamental property. It doesn't just absorb energy; it **stores** it and can then **release** or **redirect** it.
    *   **Application:**
        *   **Black Panther's Suit:** The most prominent example. Impacts, bullets, explosions – all kinetic energy is absorbed by the suit's vibranium weave. This energy is stored within the suit's internal matrix and can then be discharged as concussive blasts (the purple energy wave).
        *   **Shields & Defenses

In [ ]:
# Bonus question: Ask for a comparison that references previous answers
bonus_question = "Compare the scientific approaches of Iron Man and Black Panther. Which is more advanced?"
print(f"Question: {bonus_question}\n")

# Send the message and get the response
try:
    bonus_response = hero_chat.send_message(bonus_question)
    print(f"Gemini: {bonus_response.text}")
    print("\nNotice how the model remembers the context from previous questions and can make comparisons!")
except Exception as e:
    print(f"Error: {str(e)}")

In [ ]:
# Try your own question!
# Replace this with any superhero-related question you want to ask
your_question = "What would happen if Spider-Man and Batman teamed up?"

print(f"Your Question: {your_question}\n")

# Send the message and get the response
try:
    your_response = hero_chat.send_message(your_question)
    print(f"Gemini: {your_response.text}")
except Exception as e:
    print(f"Error: {str(e)}")

print("\nTry modifying this cell with different questions to see how the conversation continues!")

## 5. Working with Structured Data and Saving Chat History

Gemini can generate structured data like JSON, which is useful for applications that need to process the output programmatically. We'll also implement a way to save chat history to a file.

**Superhero Sidekick Metaphor**: This is like asking your sidekick to compile a standardized mission report in a specific format that can be easily integrated into your team's database. It's also like keeping a log of all your communications with your sidekick for future reference and mission debriefings.

In [22]:
# Function to request structured JSON data from Gemini
def get_structured_data(prompt, model_name=None):
    """
    Request structured JSON data from Gemini.
    
    Args:
        prompt (str): The prompt requesting structured data
        model_name (str, optional): The name of the Gemini model to use
    
    Returns:
        dict: The parsed JSON response
    """
    # Use the model from environment variable if not specified
    if model_name is None:
        model_name = GEMINI_MODEL
        
    # Add explicit instructions for JSON format
    json_prompt = f"{prompt}\n\nRespond with a valid JSON object only, no other text."
    
    # Get response from Gemini
    model = genai.GenerativeModel(model_name)
    response = model.generate_content(json_prompt)
    
    # Extract and parse the JSON
    try:
        # The response might be wrapped in code blocks, so we need to extract the JSON
        response_text = response.text
        
        # If the response contains a code block, extract its content
        if "```json" in response_text:
            json_str = response_text.split("```json")[1].split("```")[0].strip()
        elif "```" in response_text:
            json_str = response_text.split("```")[1].strip()
        else:
            json_str = response_text
            
        # Parse the JSON string
        return json.loads(json_str)
    except Exception as e:
        print(f"Error parsing JSON: {e}")
        print(f"Raw response: {response.text}")
        return None

In [23]:
import json
import datetime

# Example: Request information about superheroes
print("Example of structured data:")
prompt = "Give me information about 2 popular superheroes, including their secret identity, powers, and main nemesis. Format as a JSON array."

# Get and display the structured data
superhero_data = get_structured_data(prompt)

if superhero_data:
    print("Structured data received:")
    print(json.dumps(superhero_data, indent=2))

Example of structured data:
Structured data received:
[
  {
    "name": "Spider-Man",
    "secret_identity": "Peter Parker",
    "powers": [
      "Superhuman strength, speed, and agility",
      "Ability to cling to most surfaces",
      "Precognitive 'spider-sense' that warns of danger",
      "Regenerative healing factor (accelerated)",
      "Utilizes self-invented web-shooters to project strong, adhesive web-lines"
    ],
    "main_nemesis": "Green Goblin (Norman Osborn)"
  },
  {
    "name": "Batman",
    "secret_identity": "Bruce Wayne",
    "powers": [
      "Peak human physical and mental conditioning",
      "Master martial artist and hand-to-hand combatant",
      "Genius-level intellect and master detective",
      "Access to vast wealth and high-tech gadgets/vehicles",
      "Master of stealth, disguise, and escape artistry"
    ],
    "main_nemesis": "The Joker"
  }
]
Structured data received:
[
  {
    "name": "Spider-Man",
    "secret_identity": "Peter Parker",
    "pow

### Implementing Persistent Chat History

Now let's create a class that can save chat history to a file for future reference.

In [ ]:
# Class for persistent chat
class PersistentChat:
    def __init__(self, model_name=None, history_file='chat_history.json'):
        """
        Initialize a chat with persistent history.
        
        This class enables conversations with Gemini that are saved to disk, allowing the
        chat history to persist between sessions and be analyzed later. This is especially
        useful for applications that need to maintain context over time, track user interactions,
        or analyze conversation patterns for further improvements.
        """
        # Use the model from environment variable if not specified
        if model_name is None:
            model_name = GEMINI_MODEL
            
        self.model = genai.GenerativeModel(model_name)
        self.history_file = os.path.join('../', history_file)
        self.chat_history = []
        self.chat = self.model.start_chat(history=[])
    
    def send_message(self, message):
        """Send a message and save the response to history."""
        response = self.chat.send_message(message)
        
        # Add to history
        self.chat_history.append({"role": "user", "message": message})
        self.chat_history.append({"role": "assistant", "message": response.text})
        
        # Save history
        self.save_history()
        
        return response.text
    
    def save_history(self):
        """Save chat history to file."""
        # Create a timestamped record
        history_record = {
            "timestamp": datetime.datetime.now().isoformat(),
            "conversations": self.chat_history
        }
        
        try:
            with open(self.history_file, 'w') as f:
                json.dump(history_record, f, indent=2)
            print(f"Chat history saved to {self.history_file}")
        except Exception as e:
            print(f"Error saving chat history: {e}")



Example of persistent chat:
Chat history saved to ../superhero_chat_demo.json
Response: This is one of the most classic and fiercely debated matchups in all of comicdom, and for good reason! Both Batman and Iron Man represent the pinnacle...


### Benefits of Persisting Chat History

Saving chat history to disk provides several advantages for AI applications:

1. **Continuity Between Sessions**: Users can pick up conversations where they left off, even after closing the application.

2. **Analytics and Improvement**: 
   - Track common questions and improve responses
   - Identify patterns in user interactions
   - Use historical data to fine-tune future AI models

3. **User Experience**: 
   - Maintain context for longer, more meaningful conversations
   - Provide personalized responses based on past interactions
   - Create a "memory" that builds relationships with users

4. **Debugging and Quality Control**:
   - Review past interactions to identify where the AI might have provided incorrect information
   - Understand how conversations evolve and where they might go off-track

5. **Application Integration**: The saved JSON data can be easily integrated with databases, dashboards, or other tools for further processing.

In our superhero sidekick metaphor, this is like your sidekick keeping a detailed mission log that you can review together later, helping both of you improve your teamwork and tactics for future missions.

In [25]:

# Unlike the previous chat examples, PersistentChat saves all interactions to a file,
# allowing you to review past conversations, analyze responses, or even train future models.
print("\nExample of persistent chat:")
persistent_chat = PersistentChat(history_file='./superhero_chat_demo.json')
response1 = persistent_chat.send_message("Who would win in a fight between Batman and Iron Man, and why?")
print(f"Response: {response1[:150]}...")  # Show just the beginning for brevity


Example of persistent chat:
Chat history saved to .././superhero_chat_demo.json
Response: This is one of the classic debates in comic book history, and the answer almost always comes down to a crucial factor: **prep time.**

Let's break it ...
Chat history saved to .././superhero_chat_demo.json
Response: This is one of the classic debates in comic book history, and the answer almost always comes down to a crucial factor: **prep time.**

Let's break it ...


## 6. Conclusion: Your AI Superhero Sidekick Journey

Congratulations! You've successfully:
1. Set up your environment with the Gemini API
2. Made basic requests to generate text
3. Customized model behavior with parameters
4. Built a simple chatbot with conversation history
5. Worked with structured data responses

**Next Steps:**
- Experiment with different parameters to see how they affect the responses
- Try creating a specialized assistant for a specific domain (e.g., a superhero encyclopedia or a comic book advisor)
- Explore more advanced prompt engineering techniques
- Try integrating Gemini into a web application or other projects

**Resources:**
- [Google AI Studio](https://aistudio.google.com/)
- [Gemini API Documentation](https://ai.google.dev/docs)
- [Prompt Engineering Guide](https://ai.google.dev/docs/prompt_best_practices)

Remember that LLM capabilities are constantly evolving, so keep exploring and learning as these technologies advance!

Happy coding and AI creating!